# Importações


In [48]:
import urllib
from itertools import product
from os import getenv
from sqlalchemy import create_engine
from dotenv import load_dotenv

import numpy as np
import pandas as pd
from datetime import datetime, timedelta, time
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.express as px
from fuzzywuzzy import process
from enum import Enum


FERIADOS = pd.read_csv("../assets/feriados.csv")

### Types


In [49]:
# cSpell: disable
class BSColorsEnum(Enum):

    DANGER_COLOR = "#dc3545"

    WARNING_COLOR = "#ffc107"

    SUCCESS_COLOR = "#198754"

    GREY_500_COLOR = "#adb5bd"

    GREY_600_COLOR = "#6c757d"

    GREY_700_COLOR = "#495057"

    GREY_800_COLOR = "#343a40"

    GREY_900_COLOR = "#212529"

    PRIMARY_COLOR = "#0d6efd"

    SECONDARY_COLOR = "#6c757d"

    INFO_COLOR = "#0dcaf0"

    GRAY_COLOR = "#adb5bd"

    TEAL_COLOR = "#20c997"

    ORANGE_COLOR = "#fd7e14"

    INDIGO_COLOR = "#6610f2"

    PINK_COLOR = "#d63384"

    PURPLE_COLOR = "#6f42c1"

    GREY_400_COLOR = "#ced4da"

    SPACE_CADET_COLOR = "#282f44"

    BLUE_DELFT_COLOR = "#0d6efd"


class IndicatorType(Enum):
    PERFORMANCE = "performance"
    REPAIR = "reparo"
    EFFICIENCY = "eficiencia"


def get_color(value, max_value):
    """
    Retorna uma cor hexadecimal com base no valor fornecido e no valor máximo.

    Parâmetros:
    value (float): O valor para o qual a cor será calculada.
    max_value (float): O valor máximo possível.

    Retorna:
    str: Uma cor hexadecimal correspondente ao valor fornecido.
    """

    # Cria um mapa de cores que vai do vermelho ao verde
    cmap = plt.get_cmap("RdYlGn")

    # Normaliza o valor para um número entre 0 e 1
    normalized_value = float(value) / max_value

    # Obtém a cor correspondente do mapa de cores
    rgba_color = cmap(normalized_value)

    # Converte a cor RGBA para uma string de cor hexadecimal
    hex_color = (
        f"#{int(rgba_color[0]*255):02x}{int(rgba_color[1]*255):02x}{int(rgba_color[2]*255):02x}"
    )

    return hex_color

# Database


## Conexão com o banco de dados


In [50]:
# database/connection.py

# cSpell: disable=invalid-name
load_dotenv()


class Connection:
    """
    Class Connection
    """

    def __init__(self):
        """
        Constructor

        Args:
            user (str): user
            password (str): password
            database (str): database
            driver (str): driver
            server (str): server

        Usage:
            >>> from connection import Connection
            >>> connection = Connection()
            >>> connection.get_connection()
        """
        self.__user = "bruno.thomaz"
        self.__password = ">gn68U@X@4o8"
        self.__database = "AUTOMACAO"
        self.__driver = "{ODBC Driver 17 for SQL Server}"
        self.__server = "srv-sqlserver"

    def get_connection_automacao(self):
        """
        Get connection

        Returns:
            object: connection

        Usage:
            >>> from connection import Connection
            >>> connection = Connection()
            >>> connection.get_connection()
        """
        try:
            params = urllib.parse.quote_plus(
                f"DRIVER={self.__driver};"
                f"SERVER={self.__server};"
                f"DATABASE={self.__database};"
                f"UID={self.__user};"
                f"PWD={self.__password};"
            )
            # pylint: disable=consider-using-f-string
            conexao_automacao = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
            return conexao_automacao
        # pylint: disable=broad-except
        except Exception as error:
            print(f"Error: {error}")
            return None

## Leitura do banco de dados


In [51]:
# database/db_read.py


# cSpell: disable=invalid-name
class Read(Connection):
    """
    Class Read
    Read data from the database and return a pandas dataframe
    Create query to be executed in the database
    """

    # pylint: disable=useless-super-delegation
    def __init__(self):
        """
        Constructor
        """
        super().__init__()

    def get_automacao_data(self, query: str) -> pd.DataFrame:
        """
        Get data from database AUTOMACAO and return a pandas dataframe.

        Parameters
        ----------
        query : str
            Query to be executed in the database

        Returns
        -------
        pandas dataframe
            Dataframe with the query result
        """
        try:
            connection = self.get_connection_automacao()
            data = pd.read_sql(query, connection)
            return data
        # pylint: disable=broad-except
        except Exception as error:
            print(f"Error: {error}")
            return None

    def create_automacao_query(self, table: str, where: str = None, orderby: str = None) -> str:
        """
        Create query to be executed in the database AUTOMACAO.

        Parameters
        ----------
        table : str
            Table name
        where : str
            Where clause (optional)
        orderby : str
            Order by clause (optional)

        Returns
        -------
        str
            Query to be executed in the database
        """
        query = f"SELECT * FROM AUTOMACAO.dbo.{table}"

        if where:
            query += f" WHERE {where}"

        if orderby:
            query += f" ORDER BY {orderby}"

        return query

## Query para o banco de dados


In [52]:
# // database/get_data.py
# cSpell: disable=invalid-name
class GetData:
    """
    Essa classe é responsável por realizar a leitura dos dados do banco de dados.
    É utilizada para fazer a leitura em segundo plano, sem que o usuário perceba.
    """

    def __init__(self):
        self.db_read = Read()

    def get_data(self) -> tuple:
        """
        Realiza a leitura dos dados do banco de dados.
        Retorna na ordem: df_occ, df_info, df_cadastro
        """

        # Dia de hoje
        now = pd.to_datetime("today")

        # Encontrando primeiro dia do mês atual
        first_day = now.replace(day=1)

        # Mantendo apenas a data
        first_day = first_day.strftime("%Y-%m-%d")

        # Query para leitura dos dados de ocorrência
        query_occ = self.db_read.create_automacao_query(
            table="maquina_ocorrencia",
            where=f"data_registro >= '{first_day}'",
        )

        query_info = (
            "SELECT"
            " t1.maquina_id,"
            " (SELECT TOP 1 t2.linha FROM AUTOMACAO.dbo.maquina_cadastro t2"
            " WHERE t2.maquina_id = t1.maquina_id AND t2.data_registro <= t1.data_registro"
            " ORDER BY t2.data_registro DESC, t2.hora_registro DESC) as linha,"
            " (SELECT TOP 1 t2.fabrica FROM AUTOMACAO.dbo.maquina_cadastro t2"
            " WHERE t2.maquina_id = t1.maquina_id AND t2.data_registro <= t1.data_registro"
            " ORDER BY t2.data_registro DESC, t2.hora_registro DESC) as fabrica,"
            " t1.status,"
            " t1.turno,"
            " t1.contagem_total_ciclos,"
            " t1.contagem_total_produzido,"
            " t1.data_registro,"
            " t1.hora_registro"
            " FROM "
            " AUTOMACAO.dbo.maquina_info t1"
            f" WHERE data_registro >= '{first_day}'"
            " ORDER BY t1.data_registro DESC, t1.hora_registro DESC"
        )

        query_production = (
            "WITH aux AS ("
            " SELECT"
            " t1.maquina_id,"
            " t1.turno,"
            " t1.contagem_total_ciclos,"
            " t1.contagem_total_produzido,"
            " (SELECT TOP 1 t2.linha FROM AUTOMACAO.dbo.maquina_cadastro t2"
            " WHERE t2.maquina_id = t1.maquina_id AND"
            " DATEADD(minute, -1, CAST(t2.data_registro AS DATETIME) +"
            " CAST(t2.hora_registro AS DATETIME)) <="
            " CAST(t1.data_registro AS DATETIME) + CAST(t1.hora_registro AS DATETIME)"
            " ORDER BY t2.data_registro DESC, t2.hora_registro desc) as linha,"
            " CASE"
            " WHEN CAST(t1.hora_registro AS TIME) <= '00:01'"
            " THEN DATEADD(day, -1, CAST(t1.data_registro AS DATETIME))"
            " ELSE CAST(t1.data_registro AS DATETIME)"
            " END as data_registro_aux,"
            " CAST(t1.hora_registro AS TIME) as hora_registro"
            " FROM"
            " AUTOMACAO.dbo.maquina_info t1"
            " ), aux2 AS ("
            " SELECT *,"
            " ROW_NUMBER() OVER (PARTITION BY maquina_id, turno, CAST(data_registro_aux AS DATE)"
            " ORDER BY ABS(DATEDIFF(minute, hora_registro,"
            " CASE turno WHEN 'NOT' THEN '07:59:59'"
            " WHEN 'MAT' THEN '15:59:59'"
            " WHEN 'VES' THEN '23:59:59' END))) as rn"
            " FROM aux"
            " )"
            " SELECT"
            " maquina_id,"
            " linha,"
            " turno,"
            " contagem_total_ciclos as total_ciclos,"
            " contagem_total_produzido as total_produzido,"
            " CAST(data_registro_aux AS DATE) as data_registro,"
            " hora_registro"
            " FROM"
            " aux2"
            f" WHERE rn = 1 AND data_registro_aux >= '{first_day}'  "
            " ORDER BY data_registro DESC, linha, hora_registro DESC"
        )

        print("========== Baixando dados do DB ==========")

        # Leitura dos dados
        df_occ = self.db_read.get_automacao_data(query_occ)
        df_info = self.db_read.get_automacao_data(query_info)
        df_info_production = self.db_read.get_automacao_data(query_production)

        # Verificando se os dados foram lidos corretamente
        if df_occ.empty or df_info.empty or df_info_production.empty:
            print("====== Erro na leitura dos dados ======")
            return None, None, None

        print("Ok...")

        return df_occ, df_info, df_info_production


get_data = GetData()
df_occ, df_info, df_info_production = get_data.get_data()

========== Baixando dados do DB ==========
Ok...


## Testes de saída do banco de dados


In [53]:
df_occ.head(20)

,recno,maquina_id,motivo_id,problema,solucao,data_registro,hora_registro,usuario_id
0,3373,TMF005,03,,,2024-02-01,02:58:31,000441
1,3374,TMF002,03,,,2024-02-01,02:58:41,000441
2,3375,TMF015,03,,,2024-02-01,02:58:51,000441
3,3376,TMF011,03,,,2024-02-01,02:59:00,000441
4,3377,TMF014,03,,,2024-02-01,02:59:11,000441
5,3378,TMF009,03,,,2024-02-01,02:59:22,000441
6,3379,TMF003,03,,,2024-02-01,02:59:31,000441
7,3380,TMF001,03,,,2024-02-01,02:59:42,000441
8,3381,TMF013,03,,,2024-02-01,04:26:25,000453
9,3382,TMF007,03,,,2024-02-01,04:26:33,000453


In [54]:
df_info.head(20)

,maquina_id,linha,fabrica,status,turno,contagem_total_ciclos,contagem_total_produzido,data_registro,hora_registro
0,TMF004,9,1,false,VES,0.0,0.0,2024-02-27,17:35:21.510000
1,TMF014,8,1,true,VES,1484.0,1452.0,2024-02-27,17:35:20.510000
2,TMF006,7,1,true,VES,1440.0,1440.0,2024-02-27,17:35:19.510000
3,TMF001,6,1,false,VES,0.0,0.0,2024-02-27,17:35:18.510000
4,TMF003,5,1,true,VES,1766.0,1764.0,2024-02-27,17:35:17.510000
5,TMF011,4,1,true,VES,1518.0,1510.0,2024-02-27,17:35:16.506666
6,TMF015,3,1,true,VES,1534.0,1520.0,2024-02-27,17:35:15.506666
7,TMF002,2,1,false,VES,0.0,0.0,2024-02-27,17:35:14.506666
8,TMF005,1,1,true,VES,1530.0,1520.0,2024-02-27,17:35:13.503333
9,TMF009,14,2,false,VES,0.0,0.0,2024-02-27,17:34:34.726666


In [55]:
df_info_production.head(20)

,maquina_id,linha,turno,total_ciclos,total_produzido,data_registro,hora_registro
0,TMF013,0,MAT,0.0,0.0,2024-02-27,09:02:30.206666
1,TMF013,0,NOT,0.0,0.0,2024-02-27,07:58:30.013333
2,TMF005,1,VES,1530.0,1520.0,2024-02-27,17:35:13.503333
3,TMF005,1,MAT,5966.0,5934.0,2024-02-27,15:59:13.213333
4,TMF005,1,NOT,1578.0,1510.0,2024-02-27,07:59:11.776666
5,TMF002,2,VES,0.0,0.0,2024-02-27,17:35:14.506666
6,TMF002,2,MAT,682.0,648.0,2024-02-27,15:59:14.216666
7,TMF002,2,NOT,7664.0,7238.0,2024-02-27,07:59:12.776666
8,TMF015,3,VES,1534.0,1520.0,2024-02-27,17:35:15.506666
9,TMF015,3,MAT,5164.0,5110.0,2024-02-27,15:59:15.216666


# Limpeza de dados e análise exploratória


## Análise de dados - Clean Data


In [56]:
# service/clean_data.py


# cSpell: disable=invalid-name
class CleanData:
    def maq_info(self, info: pd.DataFrame) -> pd.DataFrame:
        """
        Processa as informações de uma máquina e retorna um DataFrame com os dados ajustados.

        Args:
            info (pd.DataFrame): DataFrame contendo as informações da máquina.

        Returns:
            pd.DataFrame: DataFrame com os dados ajustados da máquina.
        """

        # Ordenar dataframe
        df_info = info.sort_values(by=["maquina_id", "data_registro", "hora_registro", "turno"])

        # Criar coluna com data e hora unidos
        df_info["data_hora_registro"] = (
            df_info["data_registro"].astype(str)
            + " "
            + df_info["hora_registro"].astype(str).str.split(".").str[0]
        )

        # Ajustar primeira entrada se for VES
        mask = (df_info["turno"] == "VES") & (
            df_info["maquina_id"] != df_info["maquina_id"].shift()
        )
        df_info["turno"] = np.where(mask, "NOT", df_info["turno"])

        # Ajustar data_hora para pd.datetime
        df_info["data_hora_registro"] = pd.to_datetime(df_info["data_hora_registro"])

        # Ajustar horário se turno for VES - ajusta para dia anterior e horário 23:59:59
        mask = (
            (df_info["turno"] == "VES")
            & (df_info["data_hora_registro"] != df_info["data_hora_registro"].shift())
            & (df_info["data_hora_registro"].dt.time > time(0, 0, 0))
            & (df_info["data_hora_registro"].dt.time < time(0, 5, 0))
        )
        df_info["data_hora_registro"] = np.where(
            mask,
            (df_info["data_hora_registro"] - pd.Timedelta(days=1)).dt.normalize()
            + pd.Timedelta(hours=23, minutes=59, seconds=59),
            df_info["data_hora_registro"],
        )

        # Criar nova coluna status_change para identificar mudança de status
        df_info["status_change"] = df_info["status"].ne(df_info["status"].shift())

        # Criar coluna para identificar a mudança de máquina
        df_info["maquina_change"] = df_info["maquina_id"].ne(df_info["maquina_id"].shift())

        # Criar coluna para identificar a mudança de turno
        df_info["turno_change"] = df_info["turno"].ne(df_info["turno"].shift())

        # Atualizar coluna change para incluir mudança de turno
        df_info["change"] = (
            df_info["status_change"] | df_info["maquina_change"] | df_info["turno_change"]
        )

        # Agrupar por maquina e identificar data e hora da última mudança de status
        df_info["change_time"] = (
            df_info.groupby("maquina_id")["data_hora_registro"].shift(0).where(df_info["change"])
        )

        # Feito para agrupar por maquina_id e turno e manter o ultimo registro de cada grupo
        df_info = (
            df_info.groupby(["maquina_id", "change_time"])
            .agg(
                status=("status", "first"),
                turno=("turno", "first"),
                linha=("linha", "first"),
                fabrica=("fabrica", "first"),
                data_hora_registro=("data_hora_registro", "first"),
                contagem_total_ciclos=("contagem_total_ciclos", "last"),
                contagem_total_produzido=(
                    "contagem_total_produzido",
                    "last",
                ),
                change=("change", "first"),
                maquina_change=("maquina_change", "first"),
            )
            .reset_index()
        )

        # Criar nova coluna com a data_hora_final do status
        df_info["data_hora_final"] = (
            df_info.groupby("maquina_id")["data_hora_registro"]
            .shift(-1)
            .where(~df_info["maquina_change"])
        )

        # Atualizar coluna data_hora_final onde maquina_change é True
        mask = df_info["maquina_change"]
        df_info["data_hora_final"] = np.where(
            mask, df_info["change_time"].shift(-1), df_info["data_hora_final"]
        )

        # Remover colunas desnecessárias
        df_info.drop(
            columns=[
                "maquina_change",
                "change",
                "change_time",
            ],
            inplace=True,
        )

        # Remover linhas onde data_hora_final é nulo
        df_info.dropna(subset=["data_hora_final"], inplace=True)

        # Cria nova coluna tempo_registro_min para calcular o tempo de registro em minutos
        df_info["tempo_registro_min"] = (
            pd.to_datetime(df_info["data_hora_final"])
            - pd.to_datetime(df_info["data_hora_registro"])
        ).dt.total_seconds() / 60

        # Arredondar tempo_registro_min e converter para inteiro
        df_info["tempo_registro_min"] = df_info["tempo_registro_min"].round(0).astype(int)

        # Ajustar tipos
        df_info = df_info.astype(
            {
                "maquina_id": "category",
                "status": "category",
                "turno": "category",
                "linha": "category",
                "fabrica": "category",
                "tempo_registro_min": int,
                "contagem_total_ciclos": int,
                "contagem_total_produzido": int,
            }
        )

        # Ajustar nomenclatura dos status
        df_info["status"] = np.where(
            (df_info["status"] == "true") & (df_info["tempo_registro_min"] < 10),
            "in_test",
            df_info["status"],
        )
        df_info["status"] = np.where(df_info["status"] == "true", "rodando", df_info["status"])
        df_info["status"] = np.where(df_info["status"] == "false", "parada", df_info["status"])

        # Ajustar tipo do status
        df_info["status"] = df_info["status"].astype("category")

        # Ordenar pela linha e data_hora_registro
        df_info = df_info.sort_values(by=["linha", "data_hora_registro"])

        # Remover onde linha for 0
        df_info = df_info[df_info["linha"] != 0]

        # Ajustar o index
        df_info.reset_index(drop=True, inplace=True)

        return df_info

    def get_time_working(self, data: pd.DataFrame) -> pd.DataFrame:
        """
        Retorna os dados de maquina rodando.
        """

        info = self.maq_info(data)

        df_info_rodando = info[info["status"] == "rodando"]

        # Agrupar por linha, turno e data e somar o tempo de registro
        df_info_rodando = (
            df_info_rodando.groupby(
                ["linha", "turno", "status", df_info_rodando["data_hora_registro"].dt.date],
                observed=False,
            )
            .agg(tempo_registro_min=("tempo_registro_min", "sum"))
            .reset_index()
        )

        # Remover linhas onde tempo_registro_min é menor que 0
        df_info_rodando = df_info_rodando[(df_info_rodando["tempo_registro_min"] > 0)]

        # Remover onde a linha for 0
        df_info_rodando = df_info_rodando[df_info_rodando["linha"] != 0]

        # Renomear colunas
        df_info_rodando.rename(
            columns={
                "status": "motivo_nome",
                "data_hora_registro": "data_registro",
            },
            inplace=True,
        )

        # Capitalizar o motivo nome
        df_info_rodando["motivo_nome"] = df_info_rodando["motivo_nome"].str.capitalize()

        return df_info_rodando

    def get_adjusted_stops_data(self, info: pd.DataFrame) -> pd.DataFrame:
        """
        Retorna os dados de paradas ajustados de acordo com as regras definidas.

        Args:
            info (pd.DataFrame): O dataframe contendo os dados de paradas.

        Returns:
            pd.DataFrame: O dataframe com os dados de paradas ajustados.
        """
        # Certificar que data_hora_registro e data_hora_final são do tipo datetime
        info["data_hora_registro"] = pd.to_datetime(info["data_hora_registro"])
        info["data_hora_final"] = pd.to_datetime(info["data_hora_final"])

        # Ordenar por maquina_id e data_hora_registro
        df_info = info.sort_values(by=["maquina_id", "data_hora_registro"])

        # Criar coluna auxiliar para identificar a maquina rodando
        df_info["rodando"] = np.where(df_info["status"] == "rodando", 1, 0)

        # Unir grupos de paradas, levando em conta mudança de maquina e turno
        df_info["group"] = (
            (df_info["rodando"] != df_info["rodando"].shift())
            | (df_info["maquina_id"] != df_info["maquina_id"].shift())
            | (df_info["turno"] != df_info["turno"].shift())
            | (
                df_info["data_hora_registro"].dt.date
                != df_info["data_hora_registro"].shift().dt.date
            )
        ).cumsum()

        # Agrerar por grupo
        df_info = (
            df_info.groupby(["group"])
            .agg(
                maquina_id=("maquina_id", "first"),
                status=("status", "first"),
                turno=("turno", "first"),
                linha=("linha", "first"),
                fabrica=("fabrica", "first"),
                data_hora_registro=("data_hora_registro", "first"),
                data_hora_final=("data_hora_final", "last"),
                tempo_registro_min=("tempo_registro_min", "sum"),
                contagem_total_ciclos=("contagem_total_ciclos", "last"),
                contagem_total_produzido=("contagem_total_produzido", "last"),
            )
            .reset_index(drop=True)
        )

        # Alterar in_test para parada
        df_info["status"] = np.where(df_info["status"] == "in_test", "parada", df_info["status"])

        # Substituir valores nulos por np.nan
        df_info.fillna(value=np.nan, inplace=True)

        # Ordenar pela linha e data_hora_registro
        df_info = df_info.sort_values(by=["linha", "data_hora_registro"])

        # Ajusar o index
        df_info.reset_index(drop=True, inplace=True)

        return df_info

    def dayofweek_adjust(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Incluir colunas para identificar sábados, domingos e feriados.

        Args:
            df (pd.DataFrame): DataFrame com os dados de paradas.

        Returns:
            pd.DataFrame: DataFrame com as colunas adicionadas.
        """

        # Garantir que data_hora_registro é do tipo datetime
        df["data_hora_registro"] = pd.to_datetime(df["data_hora_registro"])

        # Identificar sábados
        df["sabado"] = np.where(df["data_hora_registro"].dt.dayofweek == 5, 1, 0)

        # Identificar domingos
        df["domingo"] = np.where(df["data_hora_registro"].dt.dayofweek == 6, 1, 0)

        # Ler arquivo com os feriados
        holidays = FERIADOS

        # Converter para datetime
        holidays["feriados"] = pd.to_datetime(holidays["feriados"])

        # Identificar feriados
        df["feriado"] = (
            df["data_hora_registro"]
            .dt.date.isin(pd.to_datetime(holidays["feriados"]).dt.date)
            .astype(int)
        )

        # Ordenar valores pela linha e data_hora_registro
        df = df.sort_values(by=["linha", "data_hora_registro"])

        # Remover se a linha for 0
        df = df[df["linha"] != 0]

        # Ordenar Colunas
        df = df.reindex(
            columns=[
                "fabrica",
                "linha",
                "maquina_id",
                "status",
                "turno",
                "tempo_registro_min",
                "contagem_total_produzido",
                "contagem_total_ciclos",
                "data_hora_registro",
                "data_hora_final",
                "sabado",
                "domingo",
                "feriado",
            ]
        )

        # Ajustar o index
        df.reset_index(drop=True, inplace=True)

        return df

    def get_maq_info_cleaned(self, df_info: pd.DataFrame) -> pd.DataFrame:
        """
        Retorna os dados de paradas ajustados de acordo com as regras definidas.

        Args:
            df_info (pd.DataFrame): O dataframe contendo os dados de paradas.

        Returns:
            pd.DataFrame: O dataframe com os dados de paradas ajustados.
        """

        # Ajustar dados de maquina_info
        df_info = self.maq_info(df_info)

        # Ajustar dados de paradas
        df_info = self.get_adjusted_stops_data(df_info)

        # Incluir colunas para identificar sábados, domingos e feriados
        df_info = self.dayofweek_adjust(df_info)

        return df_info

    def get_maq_occ_cleaned(self, df_occ: pd.DataFrame) -> pd.DataFrame:
        """
        Retorna os dados de ocorrências ajustados de acordo com as regras definidas.

        Args:
            df_occ (pd.DataFrame): O dataframe contendo os dados de ocorrências.

        Returns:
            pd.DataFrame: O dataframe com os dados de ocorrências ajustados.
        """

        # Motivos de Parada
        motivos = {
            1: "Ajustes",
            2: "Troca de Bobina",
            3: "Refeição",
            4: "Reunião",
            5: "Café e Ginástica Laboral",
            6: "Limpeza",
            7: "Manutenção Elétrica",
            8: "Manutenção Mecânica",
            9: "Material em Falta",
            10: "Setup de Sabor",
            11: "Setup de Tamanho",
            12: "Parada Programada",
            13: "Intervenção de Qualidade",
            14: "Linha Cheia",
            15: "Treinamento",
            16: "Limpeza Industrial",
            17: "Troca de Filme",
        }

        # Ajustar coluna motivo_id para int
        df_occ = df_occ.astype({"motivo_id": int})

        # Unir data_registro e hora_registro
        df_occ["data_hora_registro"] = (
            df_occ["data_registro"].astype(str)
            + " "
            + df_occ["hora_registro"].astype(str).str.split(".").str[0]
        )

        # Ajustar data_hora_registro para datetime
        df_occ["data_hora_registro"] = pd.to_datetime(df_occ["data_hora_registro"])

        # Criar coluna com motivo_nome com base no dicionário motivos
        df_occ["motivo_nome"] = df_occ["motivo_id"].map(motivos)

        # Ajustar "problema" e "solucao" se a string estiver vazia
        df_occ["problema"] = np.where(df_occ["problema"] == "", np.nan, df_occ["problema"])
        df_occ["solucao"] = np.where(df_occ["solucao"] == "", np.nan, df_occ["solucao"])

        # Copiar motivo_nome para problema caso problema seja nulo e motivo_id não seja 1,7,8,9,14
        df_occ["problema"] = np.where(
            (df_occ["problema"].isnull())
            & (
                ~df_occ["motivo_id"].isin(
                    [
                        1,
                        7,
                        8,
                        9,
                        14,
                    ]
                )
            ),
            df_occ["motivo_nome"],
            df_occ["problema"],
        )

        # Ajustar ordem das colunas e seus tipos
        df_occ = df_occ.reindex(
            columns=[
                "maquina_id",
                "motivo_id",
                "motivo_nome",
                "problema",
                "solucao",
                "data_hora_registro",
                "usuario_id",
            ]
        )
        df_occ = df_occ.astype(
            {
                "maquina_id": "category",
                "motivo_id": int,
                "motivo_nome": "category",
                "problema": str,
                "solucao": "category",
                "data_hora_registro": "datetime64[ns]",
                "usuario_id": "category",
            }
        )

        return df_occ

    def get_maq_production_cleaned(self, df_production: pd.DataFrame) -> pd.DataFrame:
        """
        Retorna os dados de produção ajustados de acordo com as regras definidas.

        Args:
            df_production (pd.DataFrame): O dataframe contendo os dados de produção.

        Returns:
            pd.DataFrame: O dataframe com os dados de produção ajustados.
        """

        # Incluir coluna turno_number para ordenar os turnos
        df_production["turno_number"] = df_production["turno"].map({"MAT": 2, "VES": 3, "NOT": 1})

        # Ordenar por maquina_id, data_registro e turno_number
        df_production.sort_values(by=["linha", "data_registro", "turno_number"], inplace=True)

        # Remover coluna turno_number
        df_production.drop(columns=["turno_number"], inplace=True)

        # Ajustar tipos
        df_production = df_production.astype(
            {
                "maquina_id": "category",
                "linha": "category",
                "turno": "category",
                "total_ciclos": int,
                "total_produzido": int,
                "data_registro": "datetime64[ns]",
            }
        )

        # Remover onde linha for 0
        df_production = df_production[df_production["linha"] != 0]

        # Ajustar o index
        df_production.reset_index(drop=True, inplace=True)

        return df_production


clean_data = CleanData()

## Análise de dados - Retorno de Dados


### Máquina Info


In [57]:
df_info_clean = clean_data.maq_info(df_info.copy())
df_info_clean.head(28)

,maquina_id,status,turno,linha,fabrica,data_hora_registro,contagem_total_ciclos,contagem_total_produzido,data_hora_final,tempo_registro_min
0,TMF005,parada,NOT,1,1,2024-02-01 00:01:30,0,0,2024-02-01 00:03:30,2
1,TMF005,rodando,NOT,1,1,2024-02-01 00:03:30,28,28,2024-02-01 01:37:30,94
2,TMF005,parada,NOT,1,1,2024-02-01 01:37:30,1996,1988,2024-02-01 01:39:30,2
3,TMF005,rodando,NOT,1,1,2024-02-01 01:39:30,2032,2024,2024-02-01 02:59:30,80
4,TMF005,parada,NOT,1,1,2024-02-01 02:59:30,3712,3690,2024-02-01 04:03:30,64
5,TMF005,rodando,NOT,1,1,2024-02-01 04:03:30,3744,3722,2024-02-01 05:07:31,64
6,TMF005,parada,NOT,1,1,2024-02-01 05:07:31,5100,5078,2024-02-01 05:09:31,2
7,TMF005,rodando,NOT,1,1,2024-02-01 05:09:31,5128,5104,2024-02-01 05:55:31,46
8,TMF005,parada,NOT,1,1,2024-02-01 05:55:31,6080,6052,2024-02-01 06:03:31,8
9,TMF005,rodando,NOT,1,1,2024-02-01 06:03:31,6084,6056,2024-02-01 08:01:31,118


In [58]:
df_info_stops = clean_data.get_adjusted_stops_data(df_info_clean.copy())
df_info_stops.head(28)

,maquina_id,status,turno,linha,fabrica,data_hora_registro,data_hora_final,tempo_registro_min,contagem_total_ciclos,contagem_total_produzido
0,TMF005,parada,NOT,1,1,2024-02-01 00:01:30,2024-02-01 00:03:30,2,0,0
1,TMF005,rodando,NOT,1,1,2024-02-01 00:03:30,2024-02-01 01:37:30,94,28,28
2,TMF005,parada,NOT,1,1,2024-02-01 01:37:30,2024-02-01 01:39:30,2,1996,1988
3,TMF005,rodando,NOT,1,1,2024-02-01 01:39:30,2024-02-01 02:59:30,80,2032,2024
4,TMF005,parada,NOT,1,1,2024-02-01 02:59:30,2024-02-01 04:03:30,64,3712,3690
5,TMF005,rodando,NOT,1,1,2024-02-01 04:03:30,2024-02-01 05:07:31,64,3744,3722
6,TMF005,parada,NOT,1,1,2024-02-01 05:07:31,2024-02-01 05:09:31,2,5100,5078
7,TMF005,rodando,NOT,1,1,2024-02-01 05:09:31,2024-02-01 05:55:31,46,5128,5104
8,TMF005,parada,NOT,1,1,2024-02-01 05:55:31,2024-02-01 06:03:31,8,6080,6052
9,TMF005,rodando,NOT,1,1,2024-02-01 06:03:31,2024-02-01 08:01:31,118,6084,6056


In [59]:
df_info_cleaned = clean_data.get_maq_info_cleaned(df_info.copy())
df_info_cleaned.head(28)

,fabrica,linha,maquina_id,status,turno,tempo_registro_min,contagem_total_produzido,contagem_total_ciclos,data_hora_registro,data_hora_final,sabado,domingo,feriado
0,1,1,TMF005,parada,NOT,2,0,0,2024-02-01 00:01:30,2024-02-01 00:03:30,0,0,0
1,1,1,TMF005,rodando,NOT,94,28,28,2024-02-01 00:03:30,2024-02-01 01:37:30,0,0,0
2,1,1,TMF005,parada,NOT,2,1988,1996,2024-02-01 01:37:30,2024-02-01 01:39:30,0,0,0
3,1,1,TMF005,rodando,NOT,80,2024,2032,2024-02-01 01:39:30,2024-02-01 02:59:30,0,0,0
4,1,1,TMF005,parada,NOT,64,3690,3712,2024-02-01 02:59:30,2024-02-01 04:03:30,0,0,0
5,1,1,TMF005,rodando,NOT,64,3722,3744,2024-02-01 04:03:30,2024-02-01 05:07:31,0,0,0
6,1,1,TMF005,parada,NOT,2,5078,5100,2024-02-01 05:07:31,2024-02-01 05:09:31,0,0,0
7,1,1,TMF005,rodando,NOT,46,5104,5128,2024-02-01 05:09:31,2024-02-01 05:55:31,0,0,0
8,1,1,TMF005,parada,NOT,8,6052,6080,2024-02-01 05:55:31,2024-02-01 06:03:31,0,0,0
9,1,1,TMF005,rodando,NOT,118,6056,6084,2024-02-01 06:03:31,2024-02-01 08:01:31,0,0,0


### Ocorrências


In [60]:
df_maq_occ_cleaned = clean_data.get_maq_occ_cleaned(df_occ.copy())
df_maq_occ_cleaned.head(28)

,maquina_id,motivo_id,motivo_nome,problema,solucao,data_hora_registro,usuario_id
0,TMF005,3,Refeição,Refeição,NaN,2024-02-01 02:58:31,000441
1,TMF002,3,Refeição,Refeição,NaN,2024-02-01 02:58:41,000441
2,TMF015,3,Refeição,Refeição,NaN,2024-02-01 02:58:51,000441
3,TMF011,3,Refeição,Refeição,NaN,2024-02-01 02:59:00,000441
4,TMF014,3,Refeição,Refeição,NaN,2024-02-01 02:59:11,000441
5,TMF009,3,Refeição,Refeição,NaN,2024-02-01 02:59:22,000441
6,TMF003,3,Refeição,Refeição,NaN,2024-02-01 02:59:31,000441
7,TMF001,3,Refeição,Refeição,NaN,2024-02-01 02:59:42,000441
8,TMF013,3,Refeição,Refeição,NaN,2024-02-01 04:26:25,000453
9,TMF007,3,Refeição,Refeição,NaN,2024-02-01 04:26:33,000453


### Produção


In [61]:
df_maq_info_production_cleaned = clean_data.get_maq_production_cleaned(df_info_production.copy())
df_maq_info_production_cleaned.head(28)

,maquina_id,linha,turno,total_ciclos,total_produzido,data_registro,hora_registro
0,TMF005,1,NOT,8534,8502,2024-02-01,07:59:31.586666
1,TMF005,1,MAT,5250,5206,2024-02-01,15:59:33.023333
2,TMF005,1,VES,86,0,2024-02-01,23:59:34.466666
3,TMF005,1,NOT,0,0,2024-02-02,06:45:35.683333
4,TMF005,1,VES,0,0,2024-02-02,23:58:57.400000
5,TMF005,1,NOT,0,0,2024-02-03,07:58:58.836666
6,TMF005,1,MAT,0,0,2024-02-03,15:59:00.280000
7,TMF005,1,VES,0,0,2024-02-03,23:59:01.716666
8,TMF005,1,NOT,0,0,2024-02-04,07:59:03.160000
9,TMF005,1,MAT,0,0,2024-02-04,15:59:04.600000


### Tempo Rodando


In [62]:
df_time_working = clean_data.get_time_working(df_info.copy())
df_time_working.head(28)

,linha,turno,motivo_nome,data_registro,tempo_registro_min
209,1,MAT,Rodando,2024-02-01,306
210,1,MAT,Rodando,2024-02-05,302
211,1,MAT,Rodando,2024-02-06,336
212,1,MAT,Rodando,2024-02-07,254
213,1,MAT,Rodando,2024-02-08,326
214,1,MAT,Rodando,2024-02-09,282
215,1,MAT,Rodando,2024-02-10,244
216,1,MAT,Rodando,2024-02-14,320
217,1,MAT,Rodando,2024-02-15,342
218,1,MAT,Rodando,2024-02-16,334


# Unindo informações de máquina e ocorrências


## Unindo


In [63]:
# service/join_data.py
# cSpell: disable=invalid-name


class JoinData:
    """
    Essa classe é responsável por juntar os dados de ocorrências, paradas e produção.
    """

    def join_info_occ(self, df_occ: pd.DataFrame, df_info: pd.DataFrame) -> pd.DataFrame:
        """
        Junta os dados de ocorrências e paradas.

        Args:
            df_occ (pd.DataFrame): DataFrame com os dados de ocorrências.
            df_info (pd.DataFrame): DataFrame com os dados de paradas.

        Returns:
            pd.DataFrame: DataFrame com os dados de ocorrências e paradas juntos.
        """

        # Garantir que as culunas com datas sejam datetime
        df_occ["data_hora_registro"] = pd.to_datetime(df_occ["data_hora_registro"])
        df_info["data_hora_registro"] = pd.to_datetime(df_info["data_hora_registro"])
        df_info["data_hora_final"] = pd.to_datetime(df_info["data_hora_final"])

        # Juntar os dados de ocorrências e paradas
        def merge_rows(row):
            """
            Função para juntar os dados de ocorrências e paradas.
            """
            mask = (
                (df_occ["maquina_id"] == row["maquina_id"])
                & (df_occ["data_hora_registro"] >= row["data_hora_registro"])
                & (df_occ["data_hora_registro"] <= row["data_hora_final"])
            )  # mask para identificar as paradas que ocorreram durante a ocorrência

            # Criar dataframe com as paradas que ocorreram durante a ocorrência
            if df_occ[mask].shape[0] > 0:
                return pd.Series(
                    [
                        df_occ[mask]["motivo_id"].values[0],
                        df_occ[mask]["motivo_nome"].values[0],
                        df_occ[mask]["problema"].values[0],
                        df_occ[mask]["solucao"].values[0],
                        df_occ[mask]["data_hora_registro"].values[0],
                        df_occ[mask]["usuario_id"].values[0],
                    ],
                )
            else:
                return pd.Series([None, None, None, None, None, None])

        # Aplicar a função merge_rows
        df_info[
            [
                "motivo_id",
                "motivo_nome",
                "problema",
                "solucao",
                "data_hora_registro_occ",
                "usuario_id_occ",
            ]
        ] = df_info.apply(merge_rows, axis=1)

        # Reordenar colunas
        df_info = df_info.reindex(
            columns=[
                "maquina_id",
                "linha",
                "fabrica",
                "turno",
                "data_hora_registro",
                "tempo_registro_min",
                "data_hora_final",
                "status",
                "motivo_id",
                "data_hora_registro_occ",
                "motivo_nome",
                "problema",
                "solucao",
                "usuario_id_occ",
                "contagem_total_ciclos",
                "contagem_total_produzido",
                "sabado",
                "domingo",
                "feriado",
            ]
        )

        # Ordenar por linha, data_hora_registro
        df_info.sort_values(by=["linha", "data_hora_registro"], inplace=True)

        # Ajustar problema e solução caso seja "nan"
        df_info["problema"] = df_info["problema"].astype(str)
        df_info["solucao"] = df_info["solucao"].astype(str)
        df_info["problema"] = np.where(df_info["problema"] == "nan", np.nan, df_info["problema"])
        df_info["solucao"] = np.where(df_info["solucao"] == "nan", np.nan, df_info["solucao"])

        # Remove a linha com tempo_registro_min negativo ou 0
        df_info = df_info[df_info["tempo_registro_min"] > 0]

        def move_columns(df: pd.DataFrame) -> pd.DataFrame:
            # Adiciona uma verificação para garantir que 'data_hora_registro_occ', 'data_hora_registro' e 'data_hora_final' não sejam NaN
            mask = (
                df[["data_hora_registro_occ", "data_hora_registro", "data_hora_final"]]
                .notna()
                .all(axis=1)
            )

            df["data_hora_registro_occ"] = df["data_hora_registro_occ"].replace(np.nan, pd.NaT)
            df = df.astype({"data_hora_registro_occ": "datetime64[ns]"})

            # Calcula a diferença absoluta entre data_hora_registro_occ e data_hora_registro e data_hora_final
            df["diff_registro"] = np.where(
                mask,
                (df["data_hora_registro_occ"] - df["data_hora_registro"]).dt.total_seconds().abs(),
                np.nan,
            )
            df["diff_final"] = np.where(
                mask,
                (df["data_hora_registro_occ"] - df["data_hora_final"]).dt.total_seconds().abs(),
                np.nan,
            )

            # Encontra qual das duas diferenças é menor
            mask_idxmin = df[["diff_registro", "diff_final"]].notna().any(axis=1)
            df.loc[mask & mask_idxmin, "closest"] = df.loc[
                mask & mask_idxmin, ["diff_registro", "diff_final"]
            ].idxmin(
                axis=1
            )  # removendo erro:
            # FutureWarning: The behavior of DataFrame.idxmin with all-NA values,
            # or any-NA and skipna=False, is deprecated.
            # In a future version this will raise ValueError

            columns = [
                "motivo_id",
                "motivo_nome",
                "problema",
                "solucao",
                "usuario_id_occ",
            ]

            # Move a ocorrência para a linha anterior se a data_hora_registro for mais próxima
            mask = (
                (df["closest"].shift(-1) == "diff_registro")
                & (df["status"].shift(-1) == "rodando")
                & df["motivo_id"].isnull()
            )
            for column in columns:
                df[column] = np.where(mask, df[column].shift(-1), df[column])
                df[column] = np.where(mask.shift(1), np.nan, df[column])

            # Tratamento separado para a coluna "data_hora_registro_occ"
            df["data_hora_registro_occ"] = np.where(
                mask, df["data_hora_registro_occ"].shift(-1), df["data_hora_registro_occ"]
            )
            df["data_hora_registro_occ"] = np.where(
                mask.shift(1), pd.NaT, df["data_hora_registro_occ"]
            )

            # Move a ocorrência para a linha seguinte se a data_hora_final for mais próxima
            mask = (
                (df["closest"].shift(1) == "diff_final")
                & (df["status"].shift(1) == "rodando")
                & df["motivo_id"].isnull()
            )
            for column in columns:
                df[column] = np.where(mask, df[column].shift(1), df[column])
                df[column] = np.where(mask.shift(-1), np.nan, df[column])

            # Tratamento separado para a coluna "data_hora_registro_occ"
            df["data_hora_registro_occ"] = np.where(
                mask, df["data_hora_registro_occ"].shift(1), df["data_hora_registro_occ"]
            )
            df["data_hora_registro_occ"] = np.where(
                mask.shift(-1), pd.NaT, df["data_hora_registro_occ"]
            )

            return df

        df_info = move_columns(df_info)

        # Remove colunas desnecessárias
        df_info.drop(
            columns=[
                "diff_registro",
                "diff_final",
                "closest",
            ],
            inplace=True,
        )

        # Ajustar motivo id
        df_info["motivo_id"] = df_info["motivo_id"].fillna(np.nan)

        # Ajustar em problema, solucao, usuario_id_occ e motivo_nome
        df_info["problema"] = df_info["problema"].fillna("")
        df_info["solucao"] = df_info["solucao"].fillna("")
        df_info["usuario_id_occ"] = np.where(
            df_info["usuario_id_occ"] == pd.NaT, np.nan, df_info["usuario_id_occ"]
        )
        df_info["motivo_nome"] = np.where(
            df_info["motivo_nome"] == pd.NaT, np.nan, df_info["motivo_nome"]
        )
        # Mudar de np.nan para pd.NaT em data_hora_registro_occ
        df_info["data_hora_registro_occ"] = np.where(
            df_info["data_hora_registro_occ"].isnull(), pd.NaT, df_info["data_hora_registro_occ"]
        )

        # Corrigir os formatos das colunas
        df_info = df_info.astype(
            {
                "data_hora_registro_occ": "datetime64[ns]",
                "motivo_nome": "category",
                "problema": str,
                "solucao": str,
                "usuario_id_occ": "category",
            }
        )

        # Ajustar problema para Domingo, Sábado e Feriado se motivo_id for 12
        df_info["problema"] = np.where(
            (df_info["motivo_id"] == 12) & (df_info["domingo"] == 1),
            "Domingo",
            df_info["problema"],
        )
        df_info["problema"] = np.where(
            (df_info["motivo_id"] == 12) & (df_info["sabado"] == 1),
            "Sábado",
            df_info["problema"],
        )
        df_info["problema"] = np.where(
            (df_info["motivo_id"] == 12) & (df_info["feriado"] == 1),
            "Feriado",
            df_info["problema"],
        )

        # Se o motivo_id for nulo, status for parada e o tempo registro for maior que 475 e for sábado, domingo ou feriado,
        # então o problema é "Domingo", "Sábado" ou "Feriado" e o motivo id é 12 e o motivo nome é "Parada Programada"
        condition = (
            (df_info["motivo_id"].isnull())
            & (df_info["status"] == "parada")
            & (df_info["tempo_registro_min"] > 475)
        )
        condition_sabado = condition & (df_info["sabado"] == 1)
        df_info["motivo_id"] = np.where(condition_sabado, 12, df_info["motivo_id"])
        df_info["motivo_nome"] = np.where(
            condition_sabado, "Parada Programada", df_info["motivo_nome"]
        )
        df_info["problema"] = np.where(condition_sabado, "Sábado", df_info["problema"])

        condition_domingo = condition & (df_info["domingo"] == 1)
        df_info["motivo_id"] = np.where(condition_domingo, 12, df_info["motivo_id"])
        df_info["motivo_nome"] = np.where(
            condition_domingo, "Parada Programada", df_info["motivo_nome"]
        )
        df_info["problema"] = np.where(condition_domingo, "Domingo", df_info["problema"])

        condition_feriado = condition & (df_info["feriado"] == 1)
        df_info["motivo_id"] = np.where(condition_feriado, 12, df_info["motivo_id"])
        df_info["motivo_nome"] = np.where(
            condition_feriado, "Parada Programada", df_info["motivo_nome"]
        )
        df_info["problema"] = np.where(condition_feriado, "Feriado", df_info["problema"])

        # Se o tempo de registro for maior que 480 mudar para 480
        df_info["tempo_registro_min"] = np.where(
            df_info["tempo_registro_min"] > 480, 480, df_info["tempo_registro_min"]
        )

        # Ajusta para parada programada qdo não tem o motivo e fica parada todo turno
        mask = (df_info["motivo_id"].isnull()) & (df_info["tempo_registro_min"] >= 478)
        df_info["motivo_id"] = np.where(mask, 12, df_info["motivo_id"])
        df_info["motivo_nome"] = np.where(mask, "Parada Programada", df_info["motivo_nome"])

        # ---- Busca a última parada caso não tenha motivo e seja a primeira parada do turno ---- #

        # Ordena o DataFrame por 'maquina_id' e 'turno'
        df_info.sort_values(by=["maquina_id", "data_hora_registro"], inplace=True)

        # Cria colunas temporárias com os valores do último turno
        df_info["motivo_id_last"] = df_info.groupby("maquina_id", observed=False)[
            "motivo_id"
        ].shift()
        df_info["motivo_nome_last"] = df_info.groupby("maquina_id", observed=False)[
            "motivo_nome"
        ].shift()
        df_info["problema_last"] = df_info.groupby("maquina_id", observed=False)["problema"].shift()
        df_info["solucao_last"] = df_info.groupby("maquina_id", observed=False)["solucao"].shift()

        # Cria a máscara para as linhas que atendem às condições
        mask = (df_info["tempo_registro_min"] > 200) & df_info["motivo_id"].isnull()

        # Aplica a máscara e substitui os valores nas colunas originais
        df_info.loc[mask, "motivo_id"] = df_info.loc[mask, "motivo_id_last"]
        df_info.loc[mask, "motivo_nome"] = df_info.loc[mask, "motivo_nome_last"]
        df_info.loc[mask, "problema"] = df_info.loc[mask, "problema_last"]
        df_info.loc[mask, "solucao"] = df_info.loc[mask, "solucao_last"]

        # Remove as colunas temporárias
        df_info = df_info.drop(
            columns=["motivo_id_last", "motivo_nome_last", "problema_last", "solucao_last"]
        )
        # -------------------------------------------------------------------------------- #

        # Se for motivo 12 e parada for 478 minutos ou mais, ajustar para 480
        df_info["tempo_registro_min"] = np.where(
            (df_info["motivo_id"] == 12) & (df_info["tempo_registro_min"] >= 478),
            480,
            df_info["tempo_registro_min"],
        )

        # Se o motivo for 6, o tempo de registro for maior que 200 e for sábado,
        # alterar o motivo para 16 e o motivo_nome para "Limpeza Industrial"
        m6_mask = (
            (df_info["motivo_id"] == 6)
            & (df_info["tempo_registro_min"] > 200)
            & (df_info["sabado"] == 1)
        )
        df_info["motivo_id"] = np.where(m6_mask, 16, df_info["motivo_id"])
        df_info["motivo_nome"] = np.where(
            (df_info["motivo_id"] == 16) & (df_info["sabado"] == 1),
            "Limpeza Industrial",
            df_info["motivo_nome"],
        )

        # Se for domingo e o tempo de registro for 480, ajustar para
        # motivo_id 12, motivo_nome "Parada Programada" e problema "Domingo"
        condition_mask = (df_info["domingo"] == 1) & (df_info["tempo_registro_min"] == 480)
        df_info["motivo_id"] = np.where(condition_mask, 12, df_info["motivo_id"])
        df_info["motivo_nome"] = np.where(
            condition_mask, "Parada Programada", df_info["motivo_nome"]
        )
        df_info["problema"] = np.where(condition_mask, "Domingo", df_info["problema"])

        # Remover as linhas onde a linha é 0
        df_info = df_info[df_info["linha"] != 0]

        mask_no_energy = (
            (df_info["status"] == "parada")
            & (df_info["tempo_registro_min"] < 480)
            & (df_info["status"].shift(-1) == "parada")
            & (df_info["tempo_registro_min"].shift(-1) == 480)
            & (df_info["status"].shift() == "parada")
            & (df_info["tempo_registro_min"].shift() == 480)
        )
        df_info.loc[mask_no_energy, "tempo_registro_min"] = 480

        # Remover as linhas onde status é rodando
        df_info = df_info[df_info["status"] != "rodando"]

        # Ordenar por linha e data_hora_registro
        df_info = df_info.sort_values(by=["linha", "data_hora_registro"])

        # Ajustar o index
        df_info.reset_index(drop=True, inplace=True)

        return df_info

    def problems_adjust(self, df: pd.DataFrame, threshold=88) -> pd.DataFrame:
        """
        Ajusta os problemas no DataFrame fornecido, mapeando problemas semelhantes para um nome comum.

        Args:
            df (pd.DataFrame): O DataFrame contendo os problemas a serem ajustados.
            threshold (int, opcional): O limite de similaridade para combinar problemas. Por padrão é 88.

        Returns:
            pd.DataFrame: O DataFrame com problemas ajustados.
        """
        # Encontrar problemas únicos
        unique_problems = df["problema"].unique()
        problem_mapping = {}

        # Criar um dicionário para mapear os problemas
        for problem in unique_problems:
            if problem and problem not in problem_mapping:
                problem = str(problem).capitalize()

                # Corrigir erros básicos de digitação
                problem = problem.replace("Beckup", "Backup")
                problem = problem.replace("Becukp", "Backup")
                problem = problem.replace("Stm", "Atm")

                matches = process.extract(problem, unique_problems, limit=len(unique_problems))

                # Encontrar os problemas com maior similaridade
                similar_problems = [match[0] for match in matches if match[1] >= threshold]

                # Criar um dicionário com os problemas similares
                for similar_problem in similar_problems:
                    problem_mapping[similar_problem] = problem

        # Mapear os problemas
        df["problema"] = df["problema"].map(problem_mapping)

        return df


join_data = JoinData()

## Saída de dados


In [64]:
df_info_occ = join_data.join_info_occ(df_maq_occ_cleaned.copy(), df_info_cleaned.copy())
df_info_occ

,maquina_id,linha,fabrica,turno,data_hora_registro,tempo_registro_min,data_hora_final,status,motivo_id,data_hora_registro_occ,motivo_nome,problema,solucao,usuario_id_occ,contagem_total_ciclos,contagem_total_produzido,sabado,domingo,feriado
0,TMF005,1,1,NOT,2024-02-01 00:01:30,2,2024-02-01 00:03:30,parada,NaN,NaT,NaN,,,NaN,0,0,0,0,0
1,TMF005,1,1,NOT,2024-02-01 01:37:30,2,2024-02-01 01:39:30,parada,NaN,NaT,NaN,None,,NaN,1996,1988,0,0,0
2,TMF005,1,1,NOT,2024-02-01 02:59:30,64,2024-02-01 04:03:30,parada,3.0,2024-02-01 02:58:31,Refeição,Refeição,,000441,3712,3690,0,0,0
3,TMF005,1,1,NOT,2024-02-01 05:07:31,2,2024-02-01 05:09:31,parada,NaN,NaT,NaN,None,,NaN,5100,5078,0,0,0
4,TMF005,1,1,NOT,2024-02-01 05:55:31,8,2024-02-01 06:03:31,parada,5.0,2024-02-01 05:57:06,Café e Ginástica Laboral,Café e Ginástica Laboral,,000441,6080,6052,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3347,TMF009,14,2,NOT,2024-02-26 00:01:04,480,2024-02-26 08:01:05,parada,12.0,NaT,Parada Programada,None,,NaN,0,0,0,0,0
3348,TMF009,14,2,MAT,2024-02-26 08:01:05,480,2024-02-26 16:06:30,parada,12.0,NaT,Parada Programada,None,,NaN,0,0,0,0,0
3349,TMF009,14,2,VES,2024-02-26 16:06:30,480,2024-02-27 00:02:31,parada,12.0,NaT,Parada Programada,None,,NaN,0,0,0,0,0
3350,TMF009,14,2,NOT,2024-02-27 00:02:31,480,2024-02-27 08:02:33,parada,12.0,NaT,Parada Programada,None,,NaN,0,0,0,0,0


In [65]:
df_maq_info_occ_combined = join_data.problems_adjust(df_info_occ.copy())
df_maq_info_occ_combined.head(28)

,maquina_id,linha,fabrica,turno,data_hora_registro,tempo_registro_min,data_hora_final,status,motivo_id,data_hora_registro_occ,motivo_nome,problema,solucao,usuario_id_occ,contagem_total_ciclos,contagem_total_produzido,sabado,domingo,feriado
0,TMF005,1,1,NOT,2024-02-01 00:01:30,2,2024-02-01 00:03:30,parada,NaN,NaT,NaN,NaN,,NaN,0,0,0,0,0
1,TMF005,1,1,NOT,2024-02-01 01:37:30,2,2024-02-01 01:39:30,parada,NaN,NaT,NaN,None,,NaN,1996,1988,0,0,0
2,TMF005,1,1,NOT,2024-02-01 02:59:30,64,2024-02-01 04:03:30,parada,3.0,2024-02-01 02:58:31,Refeição,Refeição,,000441,3712,3690,0,0,0
3,TMF005,1,1,NOT,2024-02-01 05:07:31,2,2024-02-01 05:09:31,parada,NaN,NaT,NaN,None,,NaN,5100,5078,0,0,0
4,TMF005,1,1,NOT,2024-02-01 05:55:31,8,2024-02-01 06:03:31,parada,5.0,2024-02-01 05:57:06,Café e Ginástica Laboral,Café e ginástica laboral,,000441,6080,6052,0,0,0
5,TMF005,1,1,MAT,2024-02-01 09:45:31,4,2024-02-01 09:49:31,parada,NaN,NaT,NaN,None,,NaN,1882,1878,0,0,0
6,TMF005,1,1,MAT,2024-02-01 10:55:32,132,2024-02-01 13:07:32,parada,3.0,2024-02-01 11:05:14,Refeição,Refeição,,000838,2700,2680,0,0,0
7,TMF005,1,1,MAT,2024-02-01 15:23:32,38,2024-02-01 16:01:33,parada,NaN,NaT,NaN,None,,NaN,5248,5206,0,0,0
8,TMF005,1,1,VES,2024-02-01 16:01:33,480,2024-02-02 00:01:34,parada,6.0,2024-02-01 16:02:50,Limpeza,Limpeza,,000939,86,0,0,0,0
9,TMF005,1,1,NOT,2024-02-02 00:01:34,480,2024-02-02 18:58:56,parada,12.0,NaT,Parada Programada,None,,NaN,0,0,0,0,0


## Dados de Saída


In [66]:
print(f"Máquina Info -> Linhas x Colunas: {df_maq_info_occ_combined.shape}")
print("----------------------------------------")
print(
    f"Máquina Info -> Memória: {df_maq_info_occ_combined.memory_usage(deep=True).sum() / 1024 ** 2:.2f} MB"
)
print("----------------------------------------")
print(f"Máquina Info -> Tipos:\n{df_maq_info_occ_combined.dtypes}")

Máquina Info -> Linhas x Colunas: (3352, 19)
----------------------------------------
Máquina Info -> Memória: 0.85 MB
----------------------------------------
Máquina Info -> Tipos:
maquina_id                        category
linha                             category
fabrica                           category
turno                             category
data_hora_registro          datetime64[ns]
tempo_registro_min                   int32
data_hora_final             datetime64[ns]
status                              object
motivo_id                          float64
data_hora_registro_occ      datetime64[ns]
motivo_nome                         object
problema                            object
solucao                             object
usuario_id_occ                    category
contagem_total_ciclos                int32
contagem_total_produzido             int32
sabado                               int32
domingo                              int32
feriado                              int32


In [67]:
print(f"Máquina Produção -> Linhas x Colunas: {df_maq_info_production_cleaned.shape}")
print("----------------------------------------")
print(
    f"Máquina Produção -> Memória: {df_maq_info_production_cleaned.memory_usage(deep=True).sum() / 1024 ** 2:.2f} MB"
)
print("----------------------------------------")
print(f"Máquina Produção -> Tipos:\n{df_maq_info_production_cleaned.dtypes}")

Máquina Produção -> Linhas x Colunas: (966, 7)
----------------------------------------
Máquina Produção -> Memória: 0.06 MB
----------------------------------------
Máquina Produção -> Tipos:
maquina_id               category
linha                    category
turno                    category
total_ciclos                int32
total_produzido             int32
data_registro      datetime64[ns]
hora_registro              object
dtype: object


# Análise de Dados - Eficiência, Performance, Reparos


## Análise de Dados


In [68]:
# service/times_data.py
# cSpell: disable=invalid-name
class TimesData:
    def __init__(self):
        # Dicionário com os descontos de parada para Eficiência

        self.desc_eff = {
            3: 60,
            5: 10,
            10: 15,
            11: 35,
            15: 60,
            17: 15,
        }

        # Dicionário com os descontos de parada para Performance

        self.desc_perf = {
            3: 60,
            5: 10,
            10: 15,
            15: 60,
            17: 15,
        }

        # Lista com os motivos de parada que não são considerados para Performance

        self.not_af_perf = [7, 8, 11, 12, 13, 16]

        # Dicionário com os descontos de parada para Reparos

        self.desc_rep = {11: 35}

        # Lista com os motivos de parada que são considerados para Reparos

        self.af_rep = [7, 8, 11]

        # Lista de Motivos que não afetam a eficiência
        self.not_af_eff = [12]

    def get_times_discount(self, info: pd.DataFrame, desc_pcp: dict[int, int]) -> pd.DataFrame:
        """
        Função para calcular os descontos de parada

        Args:
            info (pd.DataFrame): DataFrame com os dados de parada
            desc_pcp (dict[int, int]): Dicionário com os descontos de parada

        Returns:
            pd.DataFrame: DataFrame com os descontos de parada


        Exemplo:
            >>> from app.service.get_times_data import GetTimesData
            >>> import pandas as pd
            >>> get_times_data = GetTimesData()
            >>> df_times_desc = pd.dataframe()
            >>> df_result = get_times_data.get_times_discount(df_times_desc, desc_pcp)
        """

        info_stops = info.copy()

        # Adicionar coluna com descontos de parada
        info_stops["desconto_min"] = info_stops["motivo_id"].map(desc_pcp)

        # Se houver desconto, subtrair do tempo de parada e arredondar para baixo, em uma nova coluna chamada excedente
        info_stops["excedente"] = (
            info_stops["tempo_registro_min"] - info_stops["desconto_min"]
        ).clip(lower=0)

        # Se o desconto for maior que o tempo de parada, o desconto deve ser igual ao tempo de parada
        info_stops.loc[
            info_stops["desconto_min"] > info_stops["tempo_registro_min"],
            "desconto_min",
        ] = info_stops["tempo_registro_min"]

        # Criar coluna data_registro para agrupar por dia
        info_stops["data_registro"] = info_stops["data_hora_registro"].dt.date
        # Ordenar por maquina_id, data_hora_registro, turno
        info_stops.sort_values(by=["maquina_id", "data_hora_registro", "turno"], inplace=True)

        return info_stops

    def get_elapsed_time(self, turno):
        """
        Método para calcular o tempo decorrido no turno atual.

        Este método recebe o turno atual e retorna o tempo decorrido em minutos.

        Args:
            turno (str): Turno atual

        Returns:
            float: Tempo decorrido em minutos


        Exemplo:
            >>> from app.service.get_times_data import GetTimesData
            >>> import pandas as pd
            >>> get_times_data = GetTimesData()
            >>> turno = 'MAT'
            >>> tempo_decorrido = get_times_data.get_elapsed_time(turno)
        """

        now = datetime.now()

        if turno == "MAT" and 8 <= now.hour < 16:
            shift_start = now.replace(hour=8, minute=0, second=0, microsecond=0)

        elif turno == "VES" and 16 <= now.hour < 24:
            shift_start = now.replace(hour=16, minute=0, second=0, microsecond=0)

        elif turno == "NOT" and (now.hour < 8 or now.hour >= 24):
            shift_start = now.replace(hour=0, minute=0, second=0, microsecond=0)

        else:
            return 480  # retorna o tempo padrão se não estiver no turno atual

        elapsed_time = now - shift_start

        return elapsed_time.total_seconds() / 60  # retorna o tempo decorrido em minutos

    def get_eff_data(self, df_info: pd.DataFrame, df_prod: pd.DataFrame) -> pd.DataFrame:
        """
        Método para calcular os dados de eficiência.
        Este método recebe dois DataFrames, um contendo informações de tempo de eficiência e
        desconto e outro contendo informações de produção,
        e retorna um DataFrame com informações de eficiência.


        Parâmetros:

        df_info (pd.DataFrame): DataFrame contendo informações de maquina
        df_prod (pd.DataFrame): DataFrame contendo informações de produção.


        Retorna:

        pd.DataFrame: DataFrame com informações de eficiência.


        Exemplo de uso:
        ```
        times_data = TimesData()
        df_eff_times_desc = pd.dataframe()
        df_prod = pd.dataframe()
        df_result = times_data.get_eff_data(df_info, df_prod)
        ```
        """

        df_eff_times_desc = self.get_times_discount(df_info, self.desc_eff)
        df_prod_total = df_prod.copy()
        ciclo_ideal = 10.6

        # Descartar colunas desnecessárias de df_prod -> 'contagem_total_ciclos', 'usuario_id_maq_cadastro', 'data_hora_registro'
        df_prod_total.drop(columns=["total_ciclos"], inplace=True)

        # Se o motivo id não afeta a eficiência, desconto_min deve ser igual ao tempo_registro_min
        df_eff_times_desc.loc[
            df_eff_times_desc["motivo_id"].isin(self.not_af_eff), "desconto_min"
        ] = df_eff_times_desc["tempo_registro_min"]

        # Agrupar por maquina_id, data_registro e turno e o desconto
        df_eff_times_desc = (
            df_eff_times_desc.groupby(
                ["maquina_id", "linha", "data_registro", "turno"], observed=False
            )
            .agg(
                {
                    "desconto_min": "sum",
                }
            )
            .reset_index()
        )

        # Remover onde a linha for 0
        df_eff_times_desc = df_eff_times_desc[df_eff_times_desc["linha"] != 0]

        # Garantir que a coluna data_registro é datetime em ambos os dataframes
        df_eff_times_desc["data_registro"] = pd.to_datetime(
            df_eff_times_desc["data_registro"]
        ).dt.date

        df_prod_total["data_registro"] = pd.to_datetime(df_prod_total["data_registro"]).dt.date

        # Fazer merge com df_prod_total
        df_eff_times_desc = pd.merge(
            df_prod_total,
            df_eff_times_desc,
            on=["maquina_id", "linha", "turno", "data_registro"],
            how="left",
        )

        # Ajustar desc_min para 0 quando for nulo
        df_eff_times_desc.loc[df_eff_times_desc["desconto_min"].isnull(), "desconto_min"] = 0

        # Criar coluna com tempo esperado de produção
        df_eff_times_desc["tempo_esperado_min"] = df_eff_times_desc.apply(
            lambda row: (
                np.floor(self.get_elapsed_time(row["turno"]) - row["desconto_min"])
                if row["data_registro"] == datetime.now().date()
                else 480 - row["desconto_min"]
            ),
            axis=1,
        )

        # Produção esperada por turno
        df_eff_times_desc["producao_esperada"] = (
            df_eff_times_desc["tempo_esperado_min"] * ciclo_ideal
        ) * 2

        # Calcular a eficiência
        df_eff_times_desc["eficiencia"] = (
            df_eff_times_desc["total_produzido"] / df_eff_times_desc["producao_esperada"]
        )

        # Ordenar pela linha e data_registro
        df_eff_times_desc = df_eff_times_desc.sort_values(
            by=["linha", "data_registro"], ascending=True
        )

        # Remover as linhas onde a linha é 0
        df_eff_times_desc = df_eff_times_desc[df_eff_times_desc["linha"] != 0]

        # Se eficiencia for nula, substituir por 0
        df_eff_times_desc.loc[df_eff_times_desc["eficiencia"].isnull(), "eficiencia"] = 0

        # Se a produção esperada for 0 e a eficiência for 0, substituir por 1
        df_eff_times_desc.loc[
            (df_eff_times_desc["producao_esperada"] == 0),
            "eficiencia",
        ] = np.nan

        # Ajustar o index
        df_eff_times_desc.reset_index(drop=True, inplace=True)

        return df_eff_times_desc

    def get_perf_data(self, df_info: pd.DataFrame, df_prod: pd.DataFrame) -> pd.DataFrame:
        """
        Método para calcular os dados de performance.

        Este método recebe dois DataFrames, um contendo informações de máquina e
        outro contendo informações de produção,
        e retorna um DataFrame com informações de performance.

        Args:
            df_info (pd.DataFrame): DataFrame contendo informações de maquina
            df_prod (pd.DataFrame): DataFrame contendo informações de produção.

        Returns:
            pd.DataFrame: DataFrame com informações de performance.

        Exemplo de uso:
        ```
        times_data = TimesData()
        df_info = pd.dataframe()
        df_prod = pd.dataframe()
        df_result = times_data.get_perf_data(df_info, df_prod)
        ```
        """

        df_info = df_info.copy()
        df_perf_times_desc = self.get_times_discount(df_info, self.desc_perf)
        df_prod_total = df_prod.copy()

        # Descartar colunas desnecessárias de df_prod
        df_prod_total.drop(
            columns=[
                "total_ciclos",
                "total_produzido",
            ],
            inplace=True,
        )

        # Conseguir apenas as datas/turno se o motivo_id = 12 e o tempo de registro for maior que 480
        datas_programadas = df_perf_times_desc[
            (df_perf_times_desc["motivo_id"] == 12)
            & (df_perf_times_desc["tempo_registro_min"] == 480)
        ][["linha", "data_registro", "turno"]]

        # Remover as linhas que não afetam a performance
        df_perf_times_desc = df_perf_times_desc[
            ~df_perf_times_desc["motivo_id"].isin(self.not_af_perf)
        ]

        # Criar coluna 'afeta' para identificar as paradas que afetam a performance
        df_perf_times_desc["afeta"] = df_perf_times_desc["excedente"]

        # Se desconto for nulo, substituir afeta pelo valor de tempo_registro_min
        df_perf_times_desc.loc[df_perf_times_desc["desconto_min"].isnull(), "afeta"] = (
            df_perf_times_desc["tempo_registro_min"]
        )

        # Agrupar por maquina_id, data_registro e turno e somar o tempo de
        # desconto e o afeta
        df_perf_times_desc = (
            df_perf_times_desc.groupby(
                ["maquina_id", "linha", "data_registro", "turno"], observed=False
            )
            .agg(
                {
                    "desconto_min": "sum",
                    "afeta": "sum",
                }
            )
            .reset_index()
        )

        # Garantir que a coluna data_registro é datetime em ambos os dataframes
        df_perf_times_desc["data_registro"] = pd.to_datetime(
            df_perf_times_desc["data_registro"]
        ).dt.date

        df_prod_total["data_registro"] = pd.to_datetime(df_prod_total["data_registro"]).dt.date
        # Fazer merge com df_prod_total

        df_perf_times_desc = pd.merge(
            df_prod_total,
            df_perf_times_desc,
            on=["maquina_id", "linha", "turno", "data_registro"],
            how="left",
        )

        # Ajustar desconto_min para 0 quando for nulo
        df_perf_times_desc.loc[df_perf_times_desc["desconto_min"].isnull(), "desconto_min"] = 0

        # Ajustar afeta para 0 quando for nulo
        df_perf_times_desc.loc[df_perf_times_desc["afeta"].isnull(), "afeta"] = 0

        # Criar coluna com tempo esperado de produção
        df_perf_times_desc["tempo_esperado_min"] = df_perf_times_desc.apply(
            lambda row: (
                np.floor(self.get_elapsed_time(row["turno"]) - row["desconto_min"])
                if row["data_registro"] == datetime.now().date()
                else 480 - row["desconto_min"]
            ),
            axis=1,
        )

        # Calcular a performance
        df_perf_times_desc["performance"] = (
            df_perf_times_desc["afeta"] / df_perf_times_desc["tempo_esperado_min"]
        )

        # Ordenar pela linha e data_registro
        df_perf_times_desc = df_perf_times_desc.sort_values(
            by=["linha", "data_registro"], ascending=True
        )

        # Remover as linhas onde a linha é 0
        df_perf_times_desc = df_perf_times_desc[df_perf_times_desc["linha"] != 0]

        # Se a data e o turno coincidem com datas_programadas, a performance é np.nan
        # Converter 'data_registro' para datetime
        df_perf_times_desc["data_registro"] = pd.to_datetime(df_perf_times_desc["data_registro"])
        datas_programadas["data_registro"] = pd.to_datetime(datas_programadas["data_registro"])

        # Criar uma chave única em ambos os DataFrames
        df_perf_times_desc["key"] = (
            df_perf_times_desc["linha"].astype(str)
            + df_perf_times_desc["data_registro"].dt.strftime("%Y-%m-%d")
            + df_perf_times_desc["turno"].astype(str)
        )
        datas_programadas["key"] = (
            datas_programadas["linha"].astype(str)
            + datas_programadas["data_registro"].dt.strftime("%Y-%m-%d")
            + datas_programadas["turno"].astype(str)
        )

        # Se a chave coincide com datas_programadas, a performance é np.nan
        df_perf_times_desc.loc[
            df_perf_times_desc["key"].isin(datas_programadas["key"]),
            "performance",
        ] = np.nan

        # Remover a coluna 'key'
        df_perf_times_desc.drop(columns=["key"], inplace=True)
        datas_programadas.drop(columns=["key"], inplace=True)

        # Ajustar o index
        df_perf_times_desc.reset_index(drop=True, inplace=True)

        return df_perf_times_desc

    def get_repair_data(self, df_info: pd.DataFrame, df_prod: pd.DataFrame) -> pd.DataFrame:
        """
        Método para calcular os dados de reparo.

        Este método recebe dois DataFrames, um contendo informações de máquina e
        e outro contendo informações de produção,
        e retorna um DataFrame com informações de Reparo.


        ### Parâmetros:
        df_info (pd.DataFrame): DataFrame contendo informações de maquina

        df_prod (pd.DataFrame): DataFrame contendo informações de produção.

        ### Retorna:

        pd.DataFrame: DataFrame com informações de performance.

        ### Exemplo de uso:
        ```
        times_data = TimesData()
        df_info = pd.dataframe()
        df_prod = pd.dataframe()
        df_result = times_data.get_repair_data(df_info, df_prod)
        ```
        """

        df_info = df_info.copy()

        df_rep_times_desc = self.get_times_discount(df_info, self.desc_rep)

        df_prod_total = df_prod.copy()

        # Descartar colunas desnecessárias de df_prod
        df_prod_total.drop(
            columns=[
                "total_ciclos",
                "total_produzido",
            ],
            inplace=True,
        )

        # Conseguir apenas as datas que o motivo_id = 12 e o tempo de registro for maior que 480
        datas_programadas = df_rep_times_desc[
            (df_rep_times_desc["motivo_id"] == 12)
            & (df_rep_times_desc["tempo_registro_min"] == 480)
        ][["linha", "data_registro", "turno"]]

        # Remover as linhas que não afetam o reparo
        df_rep_times_desc = df_rep_times_desc[df_rep_times_desc["motivo_id"].isin(self.af_rep)]

        # Criar coluna 'afeta' para identificar as paradas que afetam o reparo
        df_rep_times_desc["afeta"] = df_rep_times_desc["excedente"]

        # Se desconto for nulo, substituir afeta pelo valor de tempo_registro_min
        df_rep_times_desc.loc[df_rep_times_desc["desconto_min"].isnull(), "afeta"] = (
            df_rep_times_desc["tempo_registro_min"]
        )

        # Agrupar por maquina_id, data_registro e turno e somar o tempo de
        # desconto e o afeta
        df_rep_times_desc = (
            df_rep_times_desc.groupby(
                ["maquina_id", "linha", "data_registro", "turno"], observed=False
            )
            .agg(
                {
                    "desconto_min": "sum",
                    "afeta": "sum",
                }
            )
            .reset_index()
        )

        # Garantir que a coluna data_registro é datetime em ambos os dataframes
        df_rep_times_desc["data_registro"] = pd.to_datetime(
            df_rep_times_desc["data_registro"]
        ).dt.date
        df_prod_total["data_registro"] = pd.to_datetime(df_prod_total["data_registro"]).dt.date

        # Fazer merge com df_prod_total
        df_rep_times_desc = pd.merge(
            df_prod_total,
            df_rep_times_desc,
            on=["maquina_id", "linha", "turno", "data_registro"],
            how="left",
        )

        # Ajustar desconto_min para 0 quando for nulo
        df_rep_times_desc.loc[df_rep_times_desc["desconto_min"].isnull(), "desconto_min"] = 0

        # Ajustar afeta para 0 quando for nulo
        df_rep_times_desc.loc[df_rep_times_desc["afeta"].isnull(), "afeta"] = 0

        # Criar coluna com tempo esperado de produção
        df_rep_times_desc["tempo_esperado_min"] = df_rep_times_desc.apply(
            lambda row: (
                np.floor(self.get_elapsed_time(row["turno"]) - row["desconto_min"])
                if row["data_registro"] == datetime.now().date()
                else 480 - row["desconto_min"]
            ),
            axis=1,
        )

        # Calcular o reparo
        df_rep_times_desc["reparo"] = (
            df_rep_times_desc["afeta"] / df_rep_times_desc["tempo_esperado_min"]
        )

        # Ordenar pela linha e data_registro
        df_rep_times_desc = df_rep_times_desc.sort_values(
            by=["linha", "data_registro"], ascending=True
        )

        # Remover as linhas onde a linha é 0
        df_rep_times_desc = df_rep_times_desc[df_rep_times_desc["linha"] != 0]

        # Se a data e o turno coincidem com datas_programadas, a performance é np.nan
        # Converter 'data_registro' para datetime
        df_rep_times_desc["data_registro"] = pd.to_datetime(df_rep_times_desc["data_registro"])
        datas_programadas["data_registro"] = pd.to_datetime(datas_programadas["data_registro"])

        # Criar uma chave única em ambos os DataFrames
        df_rep_times_desc["key"] = (
            df_rep_times_desc["linha"].astype(str)
            + df_rep_times_desc["data_registro"].dt.strftime("%Y-%m-%d")
            + df_rep_times_desc["turno"].astype(str)
        )
        datas_programadas["key"] = (
            datas_programadas["linha"].astype(str)
            + datas_programadas["data_registro"].dt.strftime("%Y-%m-%d")
            + datas_programadas["turno"].astype(str)
        )

        # Se a chave coincide com datas_programadas, a performance é np.nan
        df_rep_times_desc.loc[
            df_rep_times_desc["key"].isin(datas_programadas["key"]),
            "reparo",
        ] = np.nan

        # Remover a coluna 'key'
        df_rep_times_desc.drop(columns=["key"], inplace=True)
        datas_programadas.drop(columns=["key"], inplace=True)

        # Ajustar o index
        df_rep_times_desc.reset_index(drop=True, inplace=True)

        return df_rep_times_desc


times_data = TimesData()

## Saída de Dados


### Eficiência


In [69]:
df_eff = times_data.get_eff_data(
    df_maq_info_occ_combined.copy(), df_maq_info_production_cleaned.copy()
)
df_eff.head(28)

,maquina_id,linha,turno,total_produzido,data_registro,hora_registro,desconto_min,tempo_esperado_min,producao_esperada,eficiencia
0,TMF005,1,NOT,8502,2024-02-01,07:59:31.586666,68.0,412.0,8734.4,0.973393
1,TMF005,1,MAT,5206,2024-02-01,15:59:33.023333,60.0,420.0,8904.0,0.584681
2,TMF005,1,VES,0,2024-02-01,23:59:34.466666,0.0,480.0,10176.0,0.000000
3,TMF005,1,NOT,0,2024-02-02,06:45:35.683333,480.0,0.0,0.0,NaN
4,TMF005,1,VES,0,2024-02-02,23:58:57.400000,480.0,0.0,0.0,NaN
5,TMF005,1,NOT,0,2024-02-03,07:58:58.836666,480.0,0.0,0.0,NaN
6,TMF005,1,MAT,0,2024-02-03,15:59:00.280000,480.0,0.0,0.0,NaN
7,TMF005,1,VES,0,2024-02-03,23:59:01.716666,480.0,0.0,0.0,NaN
8,TMF005,1,NOT,0,2024-02-04,07:59:03.160000,480.0,0.0,0.0,NaN
9,TMF005,1,MAT,0,2024-02-04,15:59:04.600000,480.0,0.0,0.0,NaN


### Performance


In [70]:
df_perf = times_data.get_perf_data(
    df_maq_info_occ_combined.copy(), df_maq_info_production_cleaned.copy()
)
df_perf.head(28)

,maquina_id,linha,turno,data_registro,hora_registro,desconto_min,afeta,tempo_esperado_min,performance
0,TMF005,1,NOT,2024-02-01,07:59:31.586666,68.0,10.0,412.0,0.024272
1,TMF005,1,MAT,2024-02-01,15:59:33.023333,60.0,114.0,420.0,0.271429
2,TMF005,1,VES,2024-02-01,23:59:34.466666,0.0,480.0,480.0,1.000000
3,TMF005,1,NOT,2024-02-02,06:45:35.683333,0.0,0.0,480.0,NaN
4,TMF005,1,VES,2024-02-02,23:58:57.400000,0.0,0.0,480.0,NaN
5,TMF005,1,NOT,2024-02-03,07:58:58.836666,0.0,0.0,480.0,NaN
6,TMF005,1,MAT,2024-02-03,15:59:00.280000,0.0,0.0,480.0,NaN
7,TMF005,1,VES,2024-02-03,23:59:01.716666,0.0,0.0,480.0,NaN
8,TMF005,1,NOT,2024-02-04,07:59:03.160000,0.0,0.0,480.0,NaN
9,TMF005,1,MAT,2024-02-04,15:59:04.600000,0.0,0.0,480.0,NaN


### Reparos


In [71]:
df_reparos = times_data.get_repair_data(
    df_maq_info_occ_combined.copy(), df_maq_info_production_cleaned.copy()
)
df_reparos.head(28)

,maquina_id,linha,turno,data_registro,hora_registro,desconto_min,afeta,tempo_esperado_min,reparo
0,TMF005,1,NOT,2024-02-01,07:59:31.586666,0.0,0.0,480.0,0.000000
1,TMF005,1,MAT,2024-02-01,15:59:33.023333,0.0,0.0,480.0,0.000000
2,TMF005,1,VES,2024-02-01,23:59:34.466666,0.0,0.0,480.0,0.000000
3,TMF005,1,NOT,2024-02-02,06:45:35.683333,0.0,0.0,480.0,NaN
4,TMF005,1,VES,2024-02-02,23:58:57.400000,0.0,0.0,480.0,NaN
5,TMF005,1,NOT,2024-02-03,07:58:58.836666,0.0,0.0,480.0,NaN
6,TMF005,1,MAT,2024-02-03,15:59:00.280000,0.0,0.0,480.0,NaN
7,TMF005,1,VES,2024-02-03,23:59:01.716666,0.0,0.0,480.0,NaN
8,TMF005,1,NOT,2024-02-04,07:59:03.160000,0.0,0.0,480.0,NaN
9,TMF005,1,MAT,2024-02-04,15:59:04.600000,0.0,0.0,480.0,NaN


# Componentes


## Auxiliares - Preparação de DF's


Class DFIndicators - DF's auxiliares para a preparação dos dados.


In [72]:
class DFIndicators:
    def __init__(self, df_info_ind: pd.DataFrame, df_prod_ind: pd.DataFrame):
        self.times_data = TimesData()
        self.df_info = df_info_ind
        self.df_prod = df_prod_ind
        self.indicator_functions = {
            IndicatorType.EFFICIENCY: self.times_data.get_eff_data,
            IndicatorType.PERFORMANCE: self.times_data.get_perf_data,
            IndicatorType.REPAIR: self.times_data.get_repair_data,
        }

    # ---------------- df Heatmap ---------------- #

    def get_heatmap_data(
        self, indicator: IndicatorType, turn: str = None, main: bool = False
    ) -> pd.DataFrame:

        # Cria um dataframe vazio
        dataframe = pd.DataFrame()

        # Verifica se o indicador está no dicionário, se sim, chama a função do indicador
        if indicator in self.indicator_functions:
            dataframe = self.indicator_functions[indicator](self.df_info, self.df_prod)

        # Se o turno for diferente de nulo, filtra o dataframe
        if turn:
            dataframe = dataframe[dataframe["turno"] == turn]

        # Converter 'data_registro' para datetime
        dataframe["data_registro"] = pd.to_datetime(dataframe["data_registro"])

        # Criar coluna 'data_turno' para agrupar por dia e turno
        dataframe["data_turno"] = dataframe["data_registro"].dt.strftime("%Y-%m-%d")

        group_col = ["data_turno", "linha"]

        if main:
            group_col = ["data_turno", "turno"]

        # Agrupar por data_turno e turno e calcular a média do indicador
        df_grouped = (
            dataframe.groupby(group_col, observed=False)[indicator.value].mean().reset_index()
        )

        # ------------ dataframe com datas possíveis ------------ #
        # Obter a data de início e fim do mês
        today = datetime.now()
        start_date = today.replace(day=1).strftime("%Y-%m-%d")
        end_date = (today.replace(month=today.month % 12 + 1, day=1) - timedelta(days=1)).strftime(
            "%Y-%m-%d"
        )

        # Criar um dataframe com as datas possíveis
        all_dates = pd.date_range(start=start_date, end=end_date).strftime("%Y-%m-%d")
        all_lines = dataframe["linha"].unique()
        all_dates_lines = pd.DataFrame(list(product(all_dates, all_lines)), columns=group_col)

        # Merge com o dataframe agrupado
        df_grouped = pd.merge(all_dates_lines, df_grouped, on=group_col, how="right")

        # Se a data é futura, o indicador é np.nan
        df_grouped.loc[df_grouped["data_turno"] > today.strftime("%Y-%m-%d"), indicator.value] = (
            np.nan
        )

        # Pivotar o dataframe
        if main:
            df_pivot = df_grouped.pivot(index="turno", columns="data_turno", values=indicator.value)
            df_pivot = df_pivot.reindex(["VES", "MAT", "NOT"])
        else:
            df_grouped = df_grouped.sort_values(by=["linha", "data_turno"], ascending=True)
            df_pivot = df_grouped.pivot(index="linha", columns="data_turno", values=indicator.value)

        # Remover a linha 0
        df_pivot = df_pivot[df_pivot.index != 0]

        return df_pivot


df_to_indicators = DFIndicators(df_maq_info_occ_combined, df_maq_info_production_cleaned)

In [73]:
df_to_indicators.get_heatmap_data(IndicatorType.EFFICIENCY, main=True)

data_turno,2024-02-01,2024-02-02,2024-02-03,2024-02-04,2024-02-05,2024-02-06,2024-02-07,2024-02-08,2024-02-09,2024-02-10,...,2024-02-18,2024-02-19,2024-02-20,2024-02-21,2024-02-22,2024-02-23,2024-02-24,2024-02-25,2024-02-26,2024-02-27
turno,,,,,,,,,,,,,,,,,,,,,
VES,0.006371,0.0,NaN,NaN,0.467336,0.570420,0.673967,0.674720,0.691732,0.001856,...,0.030017,0.667839,0.705575,0.618261,0.712039,0.002319,NaN,NaN,0.014544,0.397247
MAT,0.629022,NaN,0.0,NaN,0.518982,0.492105,0.595573,0.595327,0.709123,0.643005,...,NaN,0.684150,0.748290,0.707185,0.655596,0.609348,NaN,NaN,NaN,0.498194
NOT,0.743327,NaN,0.0,NaN,0.552650,0.538220,0.736367,0.768311,0.676398,0.675461,...,NaN,0.688779,0.658864,0.681729,0.624890,0.665856,0.0,0.0,NaN,0.600779


In [74]:
df_to_indicators.get_heatmap_data(IndicatorType.PERFORMANCE, "MAT")

data_turno,2024-02-01,2024-02-02,2024-02-03,2024-02-04,2024-02-05,2024-02-06,2024-02-07,2024-02-08,2024-02-09,2024-02-10,...,2024-02-18,2024-02-19,2024-02-20,2024-02-21,2024-02-22,2024-02-23,2024-02-24,2024-02-25,2024-02-26,2024-02-27
linha,,,,,,,,,,,,,,,,,,,,,
1,0.271429,NaN,NaN,NaN,0.223810,0.195238,0.395238,0.320833,0.312195,0.419048,...,NaN,0.434043,0.185714,0.300000,0.309524,0.204762,NaN,NaN,NaN,0.185714
2,0.271429,NaN,NaN,NaN,0.242857,0.295833,0.475000,0.350000,0.247619,0.238095,...,NaN,NaN,NaN,0.637500,NaN,1.000000,NaN,NaN,NaN,0.931915
3,0.142857,NaN,NaN,NaN,0.552381,0.233333,0.702128,0.370833,0.328571,0.319048,...,NaN,0.337500,0.012500,0.458537,0.409524,0.309524,NaN,NaN,NaN,0.233333
4,0.161905,NaN,NaN,NaN,0.166667,0.138095,0.471429,0.176190,0.195833,0.271429,...,NaN,0.237500,0.358974,0.071429,0.214286,0.285714,NaN,NaN,NaN,0.095833
5,0.091667,NaN,NaN,NaN,NaN,NaN,NaN,0.037500,0.100000,0.141667,...,NaN,0.016667,0.037500,0.000000,0.433333,0.131915,NaN,NaN,NaN,0.020833
6,NaN,NaN,NaN,NaN,0.075000,0.020833,0.041667,NaN,NaN,NaN,...,NaN,0.091667,0.025000,NaN,0.120833,0.033333,NaN,NaN,NaN,0.000000
7,0.025000,NaN,NaN,NaN,0.200000,NaN,0.129167,0.054167,0.230108,0.187500,...,NaN,0.033333,0.037500,0.275000,NaN,0.033333,NaN,NaN,NaN,0.079167
8,0.145833,NaN,NaN,NaN,0.058333,0.095833,0.195833,0.041667,0.195699,0.087500,...,NaN,0.012500,0.054167,0.195699,0.079167,0.000000,NaN,NaN,NaN,0.012500
9,0.150000,NaN,NaN,NaN,0.120833,0.029167,0.041667,0.066667,0.036559,0.137500,...,NaN,NaN,NaN,0.070968,0.045833,0.470833,NaN,NaN,NaN,NaN
